In [5]:
import pandas as pd
import numpy as np
from math import inf
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import NuSVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from joblib import dump #more efficient than pickle on objects that carry large numpy arrays internally
from utils import nb_utils

In [6]:
#data = "mnist_4hp"
#data = "mnist_6hp"
data = "mlpf"
save = False

In [8]:
df_info = nb_utils.get_df_info('mlpf')
df_path = df_info['df_path']
num_epochs = df_info['num_epochs']
min_hp_idx = df_info['min_hp_idx']
max_hp_idx = df_info['max_hp_idx']
min_curve_idx = df_info['min_curve_idx']


In [9]:
df = pd.read_csv(df_path)
#df = df.iloc[:300]
if data == "mlpf":
    #remove outlier
    df = df.drop(df[df.loss_99 == df.loss_99.max()].index)
df.head()

,bin_size,dropout,lr,n_glayers_id,n_glayers_reg,output_dim,weight_decay,loss_0,loss_1,loss_2,...,loss_90,loss_91,loss_92,loss_93,loss_94,loss_95,loss_96,loss_97,loss_98,loss_99
0,64.0,0.449385,0.009435,0.0,3.0,256.0,0.000044,484.088806,459.301514,455.343811,...,447.429688,447.619354,447.439209,447.404633,447.566406,447.313049,447.398407,447.414673,447.228088,447.501160
1,64.0,0.089148,0.000039,4.0,2.0,32.0,0.001984,533.915894,530.372620,531.893738,...,529.851562,529.258301,529.493103,530.038452,529.909546,528.990479,528.480469,528.026733,528.153503,528.540161
2,16.0,0.486632,0.000276,2.0,4.0,64.0,0.000065,523.189575,465.601929,461.147034,...,450.873169,450.938354,450.910492,450.801270,450.888123,450.814850,450.916229,450.814087,450.843414,450.758148
3,32.0,0.172208,0.000042,3.0,2.0,64.0,0.000007,543.986328,484.317230,472.085449,...,448.064484,448.042389,447.977570,447.926147,447.885559,447.846802,447.796967,447.781799,447.710571,447.668335
4,32.0,0.172208,0.000042,3.0,2.0,64.0,0.000007,543.924561,484.485809,472.445312,...,448.215912,544.031372,484.386292,472.220398,467.484924,464.688690,462.955078,461.424591,460.127716,459.114685


In [10]:
known_curve = 0.25

hps = df[df.columns[min_hp_idx:max_hp_idx+1]].to_numpy()

curve = df[df.columns[min_curve_idx:min_curve_idx+int(num_epochs*known_curve)]].to_numpy()
target = df[df.columns[min_curve_idx+num_epochs-2]].to_numpy()

In [11]:
difs1, difs2 = nb_utils.finite_difs(curve)
X = np.append(np.append(np.append(hps,curve,1),difs1,1),difs2,1)
y = target

In [12]:
#scale
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y.reshape(-1, 1))

In [13]:
#split in train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(str(X_train.shape)+" "+str(X_test.shape))

(236, 79) (59, 79)


In [14]:
#instatiate and train predictor
model = NuSVR()
model.fit(X_train,y_train.ravel())

NuSVR()

In [15]:
if save:
    dump(x_scaler,"x_scaler_"+data+".joblib") 
    dump(y_scaler,"y_scaler_"+data+".joblib")
    dump(model, "model_"+data+".joblib")

In [16]:
#MSE
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred)

0.01758310553319896

In [17]:
#R^2
model.score(X_test,y_test)

0.870848317660526

In [18]:
cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2')
cvs, cvs.mean(), cvs.std()

(array([0.96472685, 0.93453304, 0.88781227, 0.91275646, 0.90083134]),
 0.9201319919969938,
 0.027083463064134023)

In [69]:
#try to optimize regressor hps

from sklearn.model_selection import cross_val_score

best, C_best, Nu_best, gamma_best = -inf, -inf, -inf, -inf
for _ in range(1000):
    C = np.exp(np.random.uniform(np.log(1e-3),np.log(10.0)))
    Nu = np.random.uniform(0,1)
    gamma = "scale"
    model = NuSVR(C=C,nu=Nu,gamma=gamma)
    cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2').mean()
    if best < cvs:
        best = cvs
        C_best, Nu_best, gamma_best = C, Nu, gamma
best, C_best, Nu_best, gamma_best

(0.9270828966116923, 2.7468828960588603, 0.2823630210735182, 'scale')

In [70]:
model = NuSVR(C=C_best,nu=Nu_best,gamma=gamma_best)
model.fit(X_train,y_train.ravel())

cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2').mean()
cvs

0.9270828966116923

In [71]:
#model.fit(X_train,y_train.ravel())
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred), model.score(X_test,y_test)

(0.015854272743861367, 0.7108226771355057)